In [37]:
import cv2
import numpy as np
from tqdm import tqdm

In [38]:
IN_DIR = "E:\\M2\\result\\pool\\gopro\\tracer_pp\\"
OUT_DIR = "E:\\M2\\result\\pool\\gopro\\"
sub_dir_list = [
    "GX010036",
    "GX010037",
    "GX010038",
    "GX010039",
    "GX010040",
]

In [39]:
import random


random.seed(1234)

def put_color_to_objects(src_img, label_table):
    label_img = np.zeros_like(src_img)
    for label in range(1, label_table.max()+1):
        label_group_index = np.where(label_table == label)
        label_img[label_group_index] = random.sample(range(255), k=3)

    return label_img

In [40]:
def area_threshold_img(label_table, stats_table):
    area = stats_table[:, -1]

    for i in range(len(area)):
        if area[i] < 1000:
            label_table[label_table == i] = 0

    return label_table

In [41]:
def preprocessing(img):
    img = img[:, :, 2]
    _, img = cv2.threshold(img, 180, 255, cv2.THRESH_BINARY)
    _, labels, stats, centroids = cv2.connectedComponentsWithStats(img)
    centroids = np.hstack([centroids, stats])
    centroids = centroids[centroids[:, -1] > 1000]

    img = put_color_to_objects(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB), area_threshold_img(labels, stats))

    return img, centroids[1:]

In [ ]:
for d in range(5):
    for i in tqdm(range(0, 5001)):
        dst, res = preprocessing(img=cv2.imread(IN_DIR + sub_dir_list[d] + "\\" + f"{i}.bmp", 1))
        cv2.imwrite(OUT_DIR + "img\\" + sub_dir_list[d] + "\\" + f"{i}.png", dst)
        np.savetxt(OUT_DIR + "pp\\" + sub_dir_list[d] + "\\" + f"pp_{i}.csv", res, delimiter=',')

  0%|          | 1/5001 [00:16<22:44:33, 16.37s/it]